In [1]:
import contacts as cts
import imessages as im

In [5]:
contact = cts.filter_contacts(firstname='ishani')[0]
contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '5107503277',
 'email': None}

In [6]:
im_contact = im.get_contacts(f"id LIKE '%{contact["phone"]}%' OR id LIKE '%{contact["email"]}%'")[0]
im_contact

{'ROWID': 60,
 'id': '+15107503277',
 'country': 'US',
 'service': 'iMessage',
 'uncanonicalized_id': '5107503277',
 'person_centric_id': None}

In [7]:
if '@' in im_contact['id']:
    contact['email'] = im_contact['id']
else:
    contact['phone'] = im_contact['id']

contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '+15107503277',
 'email': None}

---

In [1]:
import contacts as cts
import imessages as im

In [2]:
im.get_chat_participants('chat28488594168438315')

[('chat28488594168438315', '+15107503277'),
 ('chat28488594168438315', '+14085041063'),
 ('chat28488594168438315', '+19254003943'),
 ('chat28488594168438315', '+16692687993'),
 ('chat28488594168438315', '+17145929889'),
 ('chat28488594168438315', '+14088962921'),
 ('chat28488594168438315', '+12068871643'),
 ('chat28488594168438315', '+14159919063'),
 ('chat28488594168438315', '+14086562467'),
 ('chat28488594168438315', '+15105090194'),
 ('chat28488594168438315', 'adrika.sen0409@gmail.com'),
 ('chat28488594168438315', '+14696166500'),
 ('chat28488594168438315', '+15303764084')]

In [17]:
im.get_conversation_history(messages[-1])

([{'dt': '2025-09-22 09:36:14',
   'guid': '3F4C53FB-1350-455F-8B7B-95AF01694D88',
   'reply_to_guid': None,
   'sender_id': '+15107503277',
   'chat_id': None,
   'chat_name': None,
   'is_from_me': 1,
   'content': 'are u awake'},
  {'dt': '2025-09-22 10:25:57',
   'guid': 'A909CFC2-5E3B-4CBF-A78D-6475C07522A5',
   'reply_to_guid': '3F4C53FB-1350-455F-8B7B-95AF01694D88',
   'sender_id': '+15107503277',
   'chat_id': None,
   'chat_name': None,
   'is_from_me': 0,
   'content': 'yesyes'},
  {'dt': '2025-09-22 10:25:59',
   'guid': '6D06D8C1-1520-445F-A835-2AC7DD45DC95',
   'reply_to_guid': 'A909CFC2-5E3B-4CBF-A78D-6475C07522A5',
   'sender_id': '+15107503277',
   'chat_id': None,
   'chat_name': None,
   'is_from_me': 0,
   'content': 'i’ve been'},
  {'dt': '2025-09-22 10:26:07',
   'guid': '63498FBC-60A9-4939-A0ED-8B78CC7341D3',
   'reply_to_guid': '6D06D8C1-1520-445F-A835-2AC7DD45DC95',
   'sender_id': '+15107503277',
   'chat_id': None,
   'chat_name': None,
   'is_from_me': 0,
   